### Data Preparation

In [ ]:
import pandas as pd
path = "C:/Users/ppirthip/OneDrive - Intel Corporation/Speckle/"
df = pd.read_csv(path+"SNR_R5_ww51.4.csv")

In [ ]:
df.shape

In [ ]:
#check for rows will blank delta
import numpy as np
df1 = df[df["DELTA"].isnull()]
df1

In [ ]:
#make folder for dataprep if it doesn't exist
import os
dataprep_path = path + 'DataPreparation/' 
if not os.path.exists(dataprep_path):
    os.makedirs(dataprep_path)
    print("DataPreparation folder created!")

In [ ]:
df1.to_csv(dataprep_path+"Blank_delta.csv",index=False)

In [ ]:
#remove blank delta
import numpy as np
df = df[df["DELTA"].notnull()]
df

In [ ]:
df["DELTA"].value_counts()

In [ ]:
#create new column for target
import numpy as np
df["SPECKLE"] = np.where(df["DELTA"]==0,0,1)
cols = df.columns.tolist()
cols = [cols[-1]]+cols[:-1] #move speckle col to the front
df = df.reindex(columns=cols)

In [ ]:
df["SPECKLE"].value_counts()

In [ ]:
df.head()

In [ ]:
# pd.options.display.max_seq_items = 2000
# df.columns

In [ ]:
df = df.reset_index(drop=True)
df.shape

In [ ]:
# df.describe()

In [ ]:
df.to_csv(dataprep_path+"SNR_R5_ww51.4_Blankdeltarem.csv",index=False)

### 1) Check for VID duplicates (Unique VID)

In [ ]:
df.shape

In [ ]:
df["SPECKLE"].value_counts()

In [ ]:
def check_duplicates(df):
    """
    Check for duplicates in VID 
    df[DataFrame]: input df    
    """
    if df['VID'].duplicated().any():
        print("There are duplicates in VID")
        df_dup = df[df.duplicated(subset=['VID'])] #get the duplicates in df, returns repeated rows only 
        df= df[df["VID"].isin(df_dup["VID"])] #get all the duplicates in df, returns original + repeated rows 
        return df
    else:
        print("There are no duplicates in VID")
    

In [ ]:
df_dup = check_duplicates(df)
df_dup #78 rows of duplicates, 39 duplicated VIDs -> ONLY 1 VID has values, the rest all blanks

In [ ]:
df_dup.to_csv(dataprep_path+"SNR_R5_ww51.4_DuplicateVID.csv",index=False)

In [ ]:
df_dup_withval = df.loc[df['VID'] == 'M1BH477200042']
df_dup_withval

In [ ]:
def duplicates_handling(df,keep):
    """
    Remove duplicates if there is any
    df[DataFrame]: input df
    keep: {‘first’, ‘last’, False}: Determines which duplicates (if any) to keep. 
    - first : Drop duplicates except for the first occurrence. 
    - last : Drop duplicates except for the last occurrence. 
    - False : Drop all duplicates.
    """
    df = df.drop_duplicates(subset=['VID'],keep=keep) #remove duplicates from df
#     display(df)
    return df

In [ ]:
df = duplicates_handling(df,keep=False)
df = pd.concat([df, df_dup_withval], ignore_index=True)
df = duplicates_handling(df,keep="first")
df

In [ ]:
check_duplicates(df)

In [ ]:
df["SPECKLE"].value_counts()

### 2 ) NA handling (NA removal/imputation)

In [ ]:
# check column for nulls
def check_colna(df):
    """
    Check each column for nulls. Returns Feature, total null and % of null for each column
    df[DataFrame]: dataframe
    
    """
    colna_df = pd.DataFrame(columns =["Feature", "Total Null", "% of Null"])
    for col in df.columns: 
        #checking if there is any null in the column

        if df[col].isnull().sum()>0: 
            
            # if null present, total number of null in the column stores here
            total_null = df[col].isnull().sum() 
            new_row = {'Feature':col, 'Total Null':total_null, '% of Null':total_null*100/len(df)}
            #append row to the dataframe
            colna_df = colna_df.append(new_row, ignore_index=True)
            
    colna_df= colna_df.sort_values("% of Null", ascending=False)    
    return colna_df  

# check rows for nulls
def check_rowna(df,supporting_fs):  
    """
    Check each row for nulls.Returns VID, total null and % of null for each row
    df[DataFrame]: dataframe
    supporting_fs[list]: all features not used for ML except VID and SPECKLE(target)
    """
    df = df.drop(supporting_fs,axis=1)    
    colrow_df = pd.DataFrame(columns =["SPECKLE","VID", "Total Null", "% of Null"])
    for i in df.index: 
        #checking if there is any null in the row
        if df.iloc[i].isnull().sum()>0:             
            # if null present, total number of null in the row stores here
            total_null = df.iloc[i].isnull().sum() 
            new_row = {'SPECKLE':df.iloc[i,0],'VID':df.iloc[i,1], 'Total Null':total_null, '% of Null':round(total_null*100/(len(df.columns)-2),2)}
            #append row to the dataframe
            colrow_df = colrow_df.append(new_row, ignore_index=True)
            
    colrow_df= colrow_df.sort_values("% of Null", ascending=False)    
    return colrow_df   

# Drop columns based on NA threshold limit
def drop_NAcol(df,NA_limit):
    '''
    Drops columns based on proportion of NA in column
    df[DataFrame]: df
    NA_limit[float/int]: Columns with proportion of NA above NA_limit will be dropped
    '''
    threshold = len(df)*(1-NA_limit)
    df=df.dropna(axis=1, thresh=threshold)
    print(df.shape)
    return df


#### a) Check column for nulls

In [ ]:
print(df.shape)
print(df["SPECKLE"].value_counts())

#### i) check nulls in whole column 

In [ ]:
#check column na for whole df
colna_df =check_colna(df)
colna_df

In [ ]:
#check columns with 100% NA
colna_df.loc[colna_df["% of Null"] ==100]

In [ ]:
colna_df.to_csv(dataprep_path+"NA_Cols.csv",index=False)

In [ ]:
#drop columns with all NA
df = df.dropna(axis=1, how='all')
df.shape

In [ ]:
check_colna(df)

#### ii) check nulls in column for speckle only

In [ ]:
df.shape

In [ ]:
#check column na for speckle
colna_speckle_df =check_colna(df.loc[df['SPECKLE'] == 1])
colna_speckle_df

In [ ]:
len(colna_speckle_df.loc[colna_speckle_df["% of Null"] ==100])

In [ ]:
colna_speckle_df.to_csv(path+"/DataPreparation/NA_Cols_speckle.csv",index=False)

In [ ]:
col100na_speckle_df = colna_speckle_df.loc[colna_speckle_df["% of Null"] ==100]
df = df.loc[:,~df.columns.isin(col100na_speckle_df['Feature'])]
df.shape

In [ ]:
colna_speckle_df =check_colna(df.loc[df['SPECKLE'] == 1])
colna_speckle_df

#### iii) check nulls in column for non-speckle only

In [ ]:
#check column na for non-speckle
colna_nonspeckle_df =check_colna(df.loc[df['SPECKLE'] == 0])
colna_nonspeckle_df

In [ ]:
#check column na for whole df
colna_df_no100NA =check_colna(df)
colna_df_no100NA

In [ ]:
df.shape

In [ ]:
colna_df_no100NA.to_csv(path+"/DataPreparation/NA_Cols_without100NA.csv",index=False)

#### b) Drop columns based on threshold limit

In [ ]:
df = drop_NAcol(df,0.8) #drop columns with >80% NA 

In [ ]:
df.shape

In [ ]:
check_colna(df)

#### c) Check row for nulls

In [ ]:
print(df.shape)
print(df["SPECKLE"].value_counts())

In [ ]:
pd.options.display.max_seq_items = 2000
df.columns

In [ ]:
supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]', 'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS',
       'INCOMING', 'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS',
       'OUT_BITS', 'DELTA']
rowna_df = check_rowna(df,supporting_fs)
rowna_df

In [ ]:
#get rows with 100% NA
row100na_df = rowna_df.loc[rowna_df["% of Null"] ==100.00]
row100na_df

In [ ]:
row100na_df.to_csv(dataprep_path+"NA_Rows.csv",index=False)

In [ ]:
row100na_df["SPECKLE"].value_counts()

#### d) Row NA handling

In [ ]:
#drop rows with 100% NA
df = df[~df["VID"].isin(row100na_df["VID"])]
df = df.reset_index(drop=True)
df

In [ ]:
df["SPECKLE"].value_counts()

In [ ]:
df.shape

In [ ]:
#check row na
supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]', 'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS',
       'INCOMING', 'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS',
       'OUT_BITS', 'DELTA']
check_rowna(df,supporting_fs)

In [ ]:
# df = df.drop(['PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_CDYN_INDICATOR_CDYN_DATA@132110'],axis=1)

In [ ]:
df.shape

In [ ]:
#NA imputation
def NA_impute(df,imptype):
    """
    Impute NA
    df[DataFrame]:df
    imptype[string]: "mean" to impute data with mean, "median" to impute data with median
    """
    if imptype == "mean":        
        df = df.fillna(df.mean())
    if imptype == "median":
        df = df.fillna(df.median())
    return df

In [ ]:
df = NA_impute(df,imptype="median")
df

In [ ]:
check_rowna(df,supporting_fs)

### 3) Handling of negative values (Imputation/conversion)

In [ ]:
#To create a dataframe with supporting features columns (to account for string variables)
supporting_fs_df = df[["ULT@MIDAS_6261_U1", "SORTLOT", "SORTLOT7", "WAFER",
        "XLOC", "YLOC", "ULT@MIDAS_6261_U2", "IB@6261[CLASSHOT]",
        "FB@6261[CLASSHOT]","TEST RESULTS", "TEST RESULTS BITS", "TR_BITS", "INCOMING",
        "INCOMING BITS", "INC_BITS", "OUTGOING", "OUTGOING BITS", "OUT_BITS","DELTA","VID"]]

In [ ]:
supporting_fs_df.head()

In [ ]:
#To create a dataframe df_keep with IDV and HVQK token family columns
cols_to_keep=["IDV", "HVQK"]
IDV_HVQK_df = df[df.columns[df.columns.str.startswith(tuple(cols_to_keep))]]

In [ ]:
IDV_HVQK_df.head()

In [ ]:
#To create dataset with supporting features, IDV and HVQK token family columns
#Concatenating supporting_fs_df and IDV_HVQK_df along columns
df_keep = pd.concat([supporting_fs_df, IDV_HVQK_df], axis=1)

In [ ]:
df_keep.head()

In [ ]:
#To create dataframe - drop columns with IDV token family columns
df_to_convert = df.drop([col for col in df if col.startswith('IDV')], axis=1)

In [ ]:
#To create dataframe - drop columns with HVQK token family columns
df_to_convert = df_to_convert.drop([col for col in df if col.startswith('HVQK')], axis=1)

In [ ]:
#To create dataframe - drop columns with supporting features columns
df_to_convert = df_to_convert.drop(["ULT@MIDAS_6261_U1", "SORTLOT", "SORTLOT7", "WAFER",
        "XLOC", "YLOC", "ULT@MIDAS_6261_U2", "IB@6261[CLASSHOT]",
        "FB@6261[CLASSHOT]","TEST RESULTS", "TEST RESULTS BITS", "TR_BITS", "INCOMING",
        "INCOMING BITS", "INC_BITS", "OUTGOING", "OUTGOING BITS", "OUT_BITS","DELTA","VID"], axis=1)

In [ ]:
df_to_convert.head()

In [ ]:
#Convert negative columns with specific token family prefix
df_positive = df_to_convert.abs()

In [ ]:
df_positive.head()

In [ ]:
#To create dataset with some parameters converted to absolute values
#Concatenating df_keep and df_positive along columns
df = pd.concat([df_keep, df_positive], axis=1)

In [ ]:
df.head()

In [ ]:
#Checking column for -5555 values
def check_col_negative5555(df):
    """
    Check each column for -5555 values. Returns Feature, total -5555 values and % of -5555 values for each column
    df[DataFrame]: dataframe
    
    """
    col_negative5555_df = pd.DataFrame(columns =["Feature", "Total -5555 Values", "% of -5555 Values"])
    for col in df.columns: 
        #checking if there is any -5555 in the column

        if df[col].isin([-5555]).sum()>0: 
            
            # if -5555 value present, total number of -5555 value in the column stores here
            total_negative_5555 = df[col].isin([-5555]).sum() 
            new_row = {'Feature':col, 'Total -5555 Values':total_negative_5555, '% of -5555 Values':round(total_negative_5555*100/len(df),2)}
            #append row to the dataframe
            col_negative5555_df = col_negative5555_df.append(new_row, ignore_index=True)
            
    col_negative5555_df = col_negative5555_df.sort_values("% of -5555 Values", ascending=False)    
    return col_negative5555_df  

In [ ]:
#check column na for whole df
col_negative5555_df =check_col_negative5555(df)
col_negative5555_df

In [ ]:
#Checking column for -999 values
def check_col_negative999(df):
    """
    Check each column for -999 values. Returns Feature, total -999 values and % of -999 values for each column
    df[DataFrame]: dataframe
    
    """
    col_negative999_df = pd.DataFrame(columns =["Feature", "Total -999 Values", "% of -999 Values"])
    for col in df.columns: 
        #checking if there is any -999 in the column

        if df[col].isin([-999]).sum()>0: 
            
            # if -999 value present, total number of -999 value in the column stores here
            total_negative_999 = df[col].isin([-999]).sum() 
            new_row = {'Feature':col, 'Total -999 Values':total_negative_999, '% of -999 Values':round(total_negative_999*100/len(df),2)}
            #append row to the dataframe
            col_negative999_df = col_negative999_df.append(new_row, ignore_index=True)
            
    col_negative999_df = col_negative999_df.sort_values("% of -999 Values", ascending=False)    
    return col_negative999_df  

In [ ]:
#check column na for whole df
col_negative999_df =check_col_negative999(df)
col_negative999_df

In [ ]:
#Checking column for -9999 values
def check_col_negative9999(df):
    """
    Check each column for -9999 values. Returns Feature, total -9999 values and % of -9999 values for each column
    df[DataFrame]: dataframe
    
    """
    col_negative9999_df = pd.DataFrame(columns =["Feature", "Total -9999 Values", "% of -9999 Values"])
    for col in df.columns: 
        #checking if there is any -9999 in the column

        if df[col].isin([-9999]).sum()>0: 
            
            # if -9999 value present, total number of -9999 value in the column stores here
            total_negative_9999 = df[col].isin([-9999]).sum() 
            new_row = {'Feature':col, 'Total -9999 Values':total_negative_9999, '% of -9999 Values':round(total_negative_9999*100/len(df),2)}
            #append row to the dataframe
            col_negative9999_df = col_negative9999_df.append(new_row, ignore_index=True)
            
    col_negative9999_df = col_negative9999_df.sort_values("% of -9999 Values", ascending=False)    
    return col_negative9999_df  

In [ ]:
#check column na for whole df
col_negative9999_df =check_col_negative9999(df)
col_negative9999_df

In [ ]:
#Checking column for -555 values
def check_col_negative555(df):
    """
    Check each column for -555 values. Returns Feature, total -555 values and % of -555 values for each column
    df[DataFrame]: dataframe
    
    """
    col_negative555_df = pd.DataFrame(columns =["Feature", "Total -555 Values", "% of -555 Values"])
    for col in df.columns: 
        #checking if there is any -555 in the column

        if df[col].isin([-555]).sum()>0: 
            
            # if -555 value present, total number of -555 value in the column stores here
            total_negative_555 = df[col].isin([-555]).sum() 
            new_row = {'Feature':col, 'Total -555 Values':total_negative_555, '% of -555 Values':round(total_negative_555*100/len(df),2)}
            #append row to the dataframe
            col_negative555_df = col_negative555_df.append(new_row, ignore_index=True)
            
    col_negative555_df = col_negative555_df.sort_values("% of -555 Values", ascending=False)    
    return col_negative555_df  

In [ ]:
#check column na for whole df
col_negative555_df =check_col_negative555(df)
col_negative555_df

In [ ]:
#Negative value imputation - Negative 5555 imputation 
def Negative_value_impute(df,imptype):
    """
    Impute Negative value (can choose negative value to impute)
    df[DataFrame]:df
    imptype[string]: "mean" to impute data with mean, "median" to impute data with median
    """
    if imptype == "mean":        
        df = df.replace(-5555,df.mean())
    if imptype == "median":
        df = df.replace(-5555,df.median())
    return df

In [ ]:
df = Negative_value_impute(df,imptype="median")
df

In [ ]:
#Negative value imputation - Negative 999 imputation 
def Negative_value_impute(df,imptype):
    """
    Impute Negative value (can choose negative value to impute)
    df[DataFrame]:df
    imptype[string]: "mean" to impute data with mean, "median" to impute data with median
    """
    if imptype == "mean":        
        df = df.replace(-999,df.mean())
    if imptype == "median":
        df = df.replace(-999,df.median())
    return df

In [ ]:
df = Negative_value_impute(df,imptype="median")
df

In [ ]:
#Negative value imputation - Negative 9999 imputation 
def Negative_value_impute(df,imptype):
    """
    Impute Negative value (can choose negative value to impute)
    df[DataFrame]:df
    imptype[string]: "mean" to impute data with mean, "median" to impute data with median
    """
    if imptype == "mean":        
        df = df.replace(-9999,df.mean())
    if imptype == "median":
        df = df.replace(-9999,df.median())
    return df

In [ ]:
df = Negative_value_impute(df,imptype="median")
df

In [ ]:
#Negative value imputation - Negative 555 imputation 
#Negative value imputation
def Negative_value_impute(df,imptype):
    """
    Impute Negative value (can choose negative value to impute)
    df[DataFrame]:df
    imptype[string]: "mean" to impute data with mean, "median" to impute data with median
    """
    if imptype == "mean":        
        df = df.replace(-555,df.mean())
    if imptype == "median":
        df = df.replace(-555,df.median())
    return df

In [ ]:
df = Negative_value_impute(df,imptype="median")
df

In [ ]:
#To check if all negative values have been converted
#Filter only numeric data
df_numeric = df._get_numeric_data()
df_numeric

In [ ]:
#False if no negative values found in columns 
pd.set_option("max_rows", None)
print([(df_numeric < 0).all(axis=0)])

In [ ]:
#Filter columns with negative values
df_negative = df_numeric.loc[:,(df_numeric < 0).any()]
df_negative

### 4) Check and remove unary(single value) columns

In [ ]:
def unary(df):
    """
    Checks for unary columns. If there are unary columns, the unary columns will be printed and removed from the df
    df[DataFrame]: input dataframe
    """
    unarycolumns = [col for col in df.columns if len(df[col].unique())==1]
    if unarycolumns:
        print("The unary column are:",unarycolumns)
        df = df.drop(unarycolumns,axis=1)
        print("Unary columns dropped!")
        return df 
    else:
        print("There are no unary columns!")

In [ ]:
unary(df)

In [ ]:
# supporting_fs = ['SPECKLE','VID','ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
#        'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
#        'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
#        'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
#        'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
# sf_df = df[supporting_fs]
# df1 = df.drop(supporting_fs,axis=1) 
# df2 = df1.drop(['PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_CDYN_INDICATOR_CDYN_DATA@132110'],axis=1)
# df_neg = df2.loc[:,((df2 == -555).any()) | ((df2 == -999).any())]
# df_neg_sf = pd.concat([sf_df,df_neg],axis=1)
# df_neg_sf

In [ ]:
# df1.columns.difference(df2.columns)

In [ ]:
# df1['PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_CDYN_INDICATOR_CDYN_DATA@132110']

In [ ]:
# df2 = df1.drop(['PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_CDYN_INDICATOR_CDYN_DATA@132110'],axis=1)
# df_neg = df2.loc[:,((df2 == -555).any()) | ((df2 == -999).any())]

In [ ]:
# df_neg_sf = pd.concat([sf_df,df_neg],axis=1)
# df_neg_sf

In [ ]:
# df_neg_sf.to_csv(path+"/DataPreparation/Negative_Cols_555_999.csv",index=False)

### 5) Split data into train and validation datasets 

In [ ]:
from sklearn.model_selection import train_test_split
from collections import Counter
def randomsamp(df,val_size):
    """
    Split whole dataset into train and validation using random sampling
    Returns X_train, X_val, y_train, y_val
    df[DataFrame]: input dataframe    
    val_size[float]:Should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
                    Train size is complement of val size
    """
    X= df.drop(["SPECKLE"],axis=1)
    y= df["SPECKLE"] 
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = val_size,random_state=42,stratify= df["SPECKLE"])
    if type(val_size)==float:
        print("Train-val split completed with",(1-val_size)*100,"-",val_size*100,"split in train-val")
    print("Shape of X_train is:", X_train.shape)
    print("Shape of X_val is:",X_val.shape)
    print("Shape of y_train is:",y_train.shape)
    print("Shape of y_val is:",y_val.shape)
    print("Distribution of y_train:",Counter(y_train))
    print("Distribution of y_val:",Counter(y_val))
    
    X_train = X_train.reset_index(drop=True)
    X_val = X_val.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_val = y_val.reset_index(drop=True)        
           
    return X_train, X_val, y_train, y_val

def targetrandomsamp(df,speckle_test_size,nonspeckle_test_size):
    """
    Dataset is split into speckle/non-speckle first. The speckle/non-speckle datasets are then split 
    into train and validation using random sampling, followed by merging of the speckle/non-speckle  
    to return X_train, X_val, y_train, y_val
    
    df[DataFrame]: input dataframe  
    
    speckle_test_size[float/int]: Val size for speckle 
    -If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
    -If int, represents the absolute number of val samples.
    -Train size is complement of val size
    
    nonspeckle_test_size[float/int]: Val size for non-speckle 
    -If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
    -If int, represents the absolute number of val samples.
    -Train size is complement of val size
    
    """
    #split data into speckle/nonspeckle
    df_s = df.loc[df["SPECKLE"] == 1] #speckle
    df_ns = df.loc[df["SPECKLE"] == 0] #nonspeckle
    
    #split speckle/nonspeckle data into train/validation
    print("For speckle data:")
    X_train_s, X_val_s, y_train_s, y_val_s = randomsamp(df_s,val_size=speckle_test_size)
    print("\nFor non-speckle data:")
    X_train_ns, X_val_ns, y_train_ns, y_val_ns = randomsamp(df_ns,val_size=nonspeckle_test_size)
    
    #concat the speckle/non-speckle train and validation
    X_train = pd.concat([X_train_s,X_train_ns], ignore_index=True)
    X_val = pd.concat([X_val_s,X_val_ns], ignore_index=True)
    y_train = pd.concat([y_train_s,y_train_ns], ignore_index=True)
    y_val = pd.concat([y_val_s,y_val_ns], ignore_index=True)
    
    X_train = X_train.reset_index(drop=True)
    X_val = X_val.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_val = y_val.reset_index(drop=True)  
    
    print("\nFinal dataset:")
    print("Distribution of y_train:",Counter(y_train))
    print("Distribution of y_val:",Counter(y_val))
    return X_train, X_val, y_train, y_val

In [ ]:
df["SPECKLE"].value_counts()

In [ ]:
#train-test split on whole data
X_train, X_val, y_train,y_val = randomsamp(df,val_size=0.3)

In [ ]:
#train-test split on speckle/non-speckle then merge
X_train, X_val, y_train, y_val = targetrandomsamp(df,speckle_test_size=500,nonspeckle_test_size=2000)

### 6) Remove supporting features

In [ ]:
supporting_fs = ['VID','ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]', 'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS',
       'INCOMING', 'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS',
       'OUT_BITS', 'DELTA']
X_train_sf = X_train[supporting_fs] #keep supporting fs of X_train
X_val_sf = X_val[supporting_fs] #keep supporting fs of X_val
X_train = X_train.drop(supporting_fs,axis=1) #remove supporting fs from X_train
X_val = X_val.drop(supporting_fs,axis=1) #remove supporting fs from X_val

### 7) Scaling the data (Standardization/Normalization)

In [ ]:
#To scale X train and X validation (test) data using various scalers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

def scale_data(X_train,X_val,scaler_type):
    """
    Scaling X train and validation with standardization or normalization
    params:
    X_train[DataFrame]: input X train
    X_val[DataFrame]: input X validation (test)
    scaler_type[None/string]: input scaling method
    - "Standardization" for Standard Scaler
    - "Normalization" for Min Max Scaler    

    """           
    if scaler_type == "Standardization":
        scaler = StandardScaler()
    if scaler_type == "Normalization":
        scaler = MinMaxScaler()

    X_train_scaled = scaler.fit_transform(X_train)
    X_train_scaled = pd.DataFrame(X_train_scaled,columns= X_train.columns)
    X_val_scaled = scaler.transform(X_val)
    X_val_scaled = pd.DataFrame(X_val_scaled,columns= X_val.columns)
       
    return X_train_scaled, X_val_scaled

In [ ]:
#X train-test scaled using Min max Scaler
X_train_scaled, X_val_scaled = scale_data(X_train,X_val,scaler_type = "Normalization")

### 8) Handling highly correlated features (features vs features/features vs target)

#### a) features vs features

In [ ]:
import time
import numpy as np
start = time.time()

#To create correlation matrix for X_train (features)
correlation_matrix = X_train_scaled.corr().abs()

#To select the upper trigular matrix from the correlation matrix of features
upper_tri = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape),k=1).astype(np.bool))
display(upper_tri)

highly_corr_features_95 = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

end = time.time()
print ("Time elapsed:", end - start)

In [ ]:
len(highly_corr_features_95)

In [ ]:
#Display correlation matrix pairs (sorted in descending order)
correlation_matrix_stack = X_train_scaled.corr().abs()

#the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)

upper_tri_stack = (correlation_matrix_stack.where(np.triu(np.ones(correlation_matrix_stack.shape), k=1).astype(np.bool))
                  .stack()
                  .sort_values(ascending=False))

#first element of upper_tri_stack series is the pair with the biggest correlation
display(upper_tri_stack)

In [ ]:
highly_corr_features_df = pd.DataFrame (highly_corr_features_95, columns = ['Feature'])
highly_corr_features_df.to_csv(path+"/DataPreparation/highly_corr_features_95.csv",index=False)

In [ ]:
#Dropping the columns with high correlation
X_train_scaled = X_train_scaled.drop(highly_corr_features_95, axis=1)
X_train_scaled

#### b) features vs target

In [ ]:
#To create correlation matrix for X_train (features) and y_train (target)
correlation_matrix_target = X_train_scaled.corrwith(y_train, axis = 0).abs()
display(correlation_matrix_target)

In [ ]:
correlation_matrix_target_df = pd.DataFrame(correlation_matrix_target)
correlation_matrix_target_df

In [ ]:
# adding column name to the respective columns
correlation_matrix_target_df.columns =['Variables']
display(correlation_matrix_target_df)

### B) Sampling to address class imbalance

In [ ]:
Counter(y_train)

In [ ]:
from imblearn.over_sampling import SMOTE 
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter

#SMOTE
def SMOTE_sampling(X_train_scaled,y_train,over_amt,under_amt=None):
    """
    Can choose SMOTE or SMOTE + random undersampling
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    over_amt [int] : amount of synthetic data to be generated with SMOTE (speckle)
    under_amt[int/None]: int - amount of nonspeckle data after undersampling
                         None - No undersampling is done
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    if under_amt == None: #SMOTE only
        sm = SMOTE(sampling_strategy= {1:over_amt},random_state=42)
        X_s, y_s = sm.fit_resample(X_train_scaled, y_train)
        
    else: #SMOTE + random undersampling
        sm = SMOTE(sampling_strategy={1:over_amt},random_state=42)
        under = RandomUnderSampler(sampling_strategy={0:under_amt},random_state=42)
        pipeline = Pipeline(steps=[('o', sm), ('u', under)])
        X_s, y_s = pipeline.fit_resample(X_train_scaled, y_train)    
         
    print("Class distribution after sampling:", Counter(y_s))     
    print("Ratio of class distribution after sampling :",round(Counter(y_s)[0]/Counter(y_s)[1],2)) 
    
    return X_s, y_s

#BorderlineSMOTE
def BorderlineSMOTE_sampling(X_train_scaled,y_train,over_amt,under_amt=None):
    """
    Can choose BorderlineSMOTE or BorderlineSMOTE + random undersampling
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    supporting_fs[list] : list of supporting features to be removed from X_train_scaled
    over_amt [int] : amount of synthetic data to be generated with BorderlineSMOTE
    under_amt[int/None]: int - amount of nonspeckle data after undersampling
                         None - No undersampling is done
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    if under_amt == None: #BorderlineSMOTE only
        bsm = BorderlineSMOTE(sampling_strategy= {1:over_amt},random_state=42)
        X_bs, y_bs = bsm.fit_resample(X_train_scaled, y_train)
        
    else: #BorderlineSMOTE + random undersampling
        bsm = BorderlineSMOTE(sampling_strategy={1:over_amt},random_state=42)
        under = RandomUnderSampler(sampling_strategy={0:under_amt},random_state=42)
        pipeline = Pipeline(steps=[('o', bsm), ('u', under)])
        X_bs, y_bs = pipeline.fit_resample(X_train_scaled, y_train)    
         
    print("Class distribution after sampling:", Counter(y_bs))     
    print("Ratio of class distribution after sampling :",round(Counter(y_bs)[0]/Counter(y_bs)[1],2)) 
    
    return X_bs, y_bs

#ADASYN
def ADASYN_sampling(X_train_scaled,y_train,over_amt,under_amt=None):
    """
    Can choose ADASYN or ADASYN + random undersampling
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    supporting_fs[list] : list of supporting features to be removed from X_train_scaled
    over_amt [int] : amount of synthetic data to be generated with ADASYN
    under_amt[int/None]: int - amount of nonspeckle data after undersampling
                         None - No undersampling is done
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    if under_amt == None: #ADASYN only
        ad = ADASYN(sampling_strategy= {1:over_amt},random_state=42)
        X_a, y_a = ad.fit_resample(X_train_scaled, y_train)
        
    else: #ADASYN + random undersampling
        ad = ADASYN(sampling_strategy={1:over_amt},random_state=42)
        under = RandomUnderSampler(sampling_strategy={0:under_amt},random_state=42)
        pipeline = Pipeline(steps=[('o', ad), ('u', under)])
        X_a, y_a = pipeline.fit_resample(X_train_scaled, y_train)    
         
    print("Class distribution after sampling:", Counter(y_a))     
    print("Ratio of class distribution after sampling :",round(Counter(y_a)[0]/Counter(y_a)[1],2)) 
    
    return X_a, y_a

In [ ]:
#SMOTE
X_s, y_s = SMOTE_sampling(X_train_scaled,y_train,over_amt=2000,under_amt=2000)      

In [ ]:
#BorderlineSMOTE
X_bs, y_bs = BorderlineSMOTE_sampling(X_train_scaled,y_train,over_amt=2000,under_amt=2000)      

In [ ]:
#ADASYN
X_a, y_a = ADASYN_sampling(X_train_scaled,y_train,over_amt=2000,under_amt=None)      

In [ ]:
#create new column for target
# import numpy as np
# df["SPECKLE"] = np.where(df["DELTA"]==0,0,1)
# cols = df.columns.tolist()
# cols = [cols[-1]]+cols[:-1] #move speckle col to the front
# df = df.reindex(columns=cols)

In [ ]:
# df.to_csv(path+"SNR_R2_ww35.2_Speckle.csv",index=False)

In [ ]:
# supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
#        'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
#        'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
#        'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
#        'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
# df1 = df.drop(supporting_fs,axis=1) 
# df1.head()

### 3) Feature Selection

In [ ]:
#make folder for featureselection if it doesn't exist
import os
featuresel_path = path + 'FeatureSelection/' 
if not os.path.exists(featuresel_path):
    os.makedirs(featuresel_path)
    print("FeatureSelection folder created!")

#### a) ANOVA

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

def anova_fs(X_train_scaled,y_train):
    """
    Anova feature selection. Return the top k features 
    X_train_scaled[df]: X of train data
    y_train[df]: y of train data (target)
    num_top_fs[int/"all"]: int -> specify the number of top fs to be returned,
                                  returns top fs and their scores in descending order
                           "all" -> returns all fs and their scores in descending order
    """
    #Select k features with top Fisher scores
    anova = SelectKBest(f_classif, k = "all")    
    anova.fit(X_train_scaled, y_train)
            
    #get the top feature names
    fs_names = X_train_scaled.columns.values[anova.get_support()] 

    #get the top feature scores
    scores = anova.scores_[anova.get_support()] 
    
    #dataframe with feature name and score in descending order
    names_scores = list(zip(fs_names, scores))
    ns_df = pd.DataFrame(data = names_scores, columns=['Feature', 'Scores'])
    ns_df = ns_df.sort_values(by=['Scores'], ascending=False)    
    ns_df['Scores_Scaled'] = ns_df['Scores']/ns_df['Scores'].sum()
    ns_df['Scores_Scaled_Cumulative %'] = ns_df['Scores_Scaled'].cumsum()*100
    ns_df = ns_df.reset_index(drop=True)
    
    return anova, ns_df

In [ ]:
anova,ns_df = anova_fs(X_train_scaled,y_train)
print(anova)
ns_df

In [ ]:
ns_df.to_csv(featuresel_path+"Anova_Fs.csv",index=False)

In [ ]:
#take top n important features only in train and validation
n=200
features = ns_df.head(n)[["Feature"]] #choose the top n important features
X_train_scaled = X_train_scaled[features["Feature"]] #filter X_train_scaled with top n important features
X_val_scaled = X_val_scaled[features["Feature"]] #filter X_val_scaled with top n important features

### b) Boruta

In [ ]:
X_train_scaled.shape

In [ ]:
y_train.value_counts()

In [ ]:
%%time

# Build the grid search on random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time

param_grid = {
#     'n_estimators': [100, 300], # number of trees, default is 100 -> not used as Boruta will determine
    'max_depth': [3,5,7], #max_depth of the tree is advised on the Boruta Github page to be between 3 to 7
    'max_features': ['auto','log2'], #number of features to consider when looking for the best split, default auto
    'min_samples_leaf': [1,100], # minimum sample leaf (decision tree end node) size, default is 1
    'class_weight': [None,'balanced'], #weights for unbalanced data     
    'random_state': [42]
}

rf_model = RandomForestClassifier() 
grid = GridSearchCV(estimator=rf_model, param_grid=param_grid,scoring='balanced_accuracy',n_jobs = -1,verbose=2,cv = 5)
grid_results = grid.fit(X_train_scaled,y_train) 

print("Best parameters:", grid_results.best_params_)

#model with best parameters
model = grid_results.best_estimator_ 
print(model)

fs_importance= pd.DataFrame(list(zip(X_train_scaled.columns, model.feature_importances_)), columns=['Feature', 'Importance']).sort_values(by=['Importance'], ascending=False)  
display(fs_importance)

In [ ]:
results_df = pd.DataFrame(grid_results.cv_results_)
results_df= results_df.drop(["mean_fit_time","std_fit_time","mean_score_time","std_score_time","split0_test_score","split1_test_score","split2_test_score","split3_test_score","split4_test_score"],axis=1)
results_df

In [ ]:
rf_fs_importance.to_csv(featuresel_path+"RandomForest.csv",index=False)

In [ ]:
# rf_fs_importance["Importance"].sum()

In [ ]:
%%time
# Creating a BorutaPy object with RandomForestClassifier as the estimator and ranking the features
from boruta import BorutaPy
boruta_selector = BorutaPy(best_rf_model, n_estimators='auto', verbose=2, perc = 90,max_iter = 100, random_state=42)                                                      
boruta_selector.fit(np.array(X_train_scaled), np.array(y_train))

In [ ]:
print(boruta_selector)
print("No. of significant features: ", boruta_selector.n_features_) 
selected_boruta_fs = pd.DataFrame({'Feature':list(X_train_scaled.columns),'Ranking':boruta_selector.ranking_}).sort_values(by='Ranking')
selected_boruta_fs = selected_boruta_fs[selected_boruta_fs['Ranking']==1]
selected_boruta_fs = selected_boruta_fs[["Feature"]]
selected_boruta_fs_imp = rf_fs_importance.loc[rf_fs_importance['Feature'].isin(selected_boruta_fs['Feature']),:]
selected_boruta_fs_imp

In [ ]:
selected_boruta_fs_imp.to_csv(featuresel_path+"Boruta.csv",index=False)

In [ ]:
#take important features only in train and validation
X_train_scaled = X_train_scaled[selected_boruta_fs_imp["Feature"]]
X_val_scaled = X_val_scaled[selected_boruta_fs_imp["Feature"]]

### c) Random Forest

In [ ]:
X_train_scaled.shape

In [ ]:
y_train.value_counts()

In [ ]:
# Build the grid search
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time

start = time.time()

param_grid = {
    'n_estimators': [100, 300], # number of trees, default is 100
    'max_features': ['auto','log2'], #number of features to consider when looking for the best split, default auto
    'min_samples_leaf': [1,100], # minimum sample leaf (decision tree end node) size, default is 1
    'class_weight': [None,'balanced'], #weights for unbalanced data     
    'random_state': [42]
}

# Grid search on random forest
rf_model = RandomForestClassifier() 
grid = GridSearchCV(estimator=rf_model, param_grid=param_grid,scoring='balanced_accuracy',n_jobs = -1,verbose=2,cv = 5)
grid_results = grid.fit(X_train_scaled,y_train) 


# # Print the time spend and number of models ran
# print("GridSearchCV took %.2f seconds for %d candidate parameter settings." % ((time.time() - start), len(grid_results.cv_results_['params'])))

end = time.time()
print ("Time elapsed:", end - start)

In [ ]:
print("Best parameters:", grid_results.best_params_)
#model with best parameters
best_rf_model = grid_results.best_estimator_ 
best_rf_model

In [ ]:
rf_fs_importance= pd.DataFrame(list(zip(X_train_scaled.columns, best_rf_model.feature_importances_)), columns=['Feature', 'Importance']).sort_values(by=['Importance'], ascending=False)  
rf_fs_importance

In [ ]:
results_df = pd.DataFrame(grid_results.cv_results_)
results_df= results_df.drop(["mean_fit_time","std_fit_time","mean_score_time","std_score_time","split0_test_score","split1_test_score","split2_test_score","split3_test_score","split4_test_score"],axis=1)
results_df

### d) XGBoost

In [ ]:
# Build the grid search
from xgboost import XGBClassifier

start = time.time()

param_grid = {
    'n_estimators': [100, 600],
    'learning_rate': [0.01],
    'min_child_weight': [2, 7],
    'gamma': [0.5, 4],
    'subsample': [0.5, 1.0],
    'colsample_bytree': [0.5, 1.0],
    'max_depth': [3, 5],
    'objective': ['binary:logistic'],
    'nthread': [1],
    'verbosity': [0],
    'random_state': [42]
}

# Grid search on XGBoost
xgb_model = XGBClassifier()
grid = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv = 5)
grid_results = grid.fit(X_train_scaled,y_train) 

# # Print the time spend and number of models ran
# print("GridSearchCV took %.2f seconds for %d candidate parameter settings." % ((time.time() - start), len(grid_results.cv_results_['params'])))

end = time.time()
print ("Time elapsed:", end - start)

In [ ]:
print("Best parameters:", grid_results.best_params_)
#model with best parameters
best_xgb_model = grid_results.best_estimator_ 
best_xgb_model

In [ ]:
xgb_fs_importance = pd.DataFrame(list(zip(X_train_scaled.columns, best_xgb_model.feature_importances_)), columns=['Feature', 'Importance']).sort_values(by=['Importance'], ascending=False)  
xgb_fs_importance

In [ ]:
results_df = pd.DataFrame(grid_results.cv_results_)
results_df= results_df.drop(["mean_fit_time","std_fit_time","mean_score_time","std_score_time","split0_test_score","split1_test_score","split2_test_score","split3_test_score","split4_test_score"],axis=1)
results_df

### f) Ensemble feature selection

In [ ]:
best_rf_model

In [ ]:
best_xgb_model

In [ ]:
%%time
anova._estimator_type = "classifier"
from xgboost import XGBClassifier
best_xgb_model = XGBClassifier(random_state = 42)

#ensemble feature selection
from sklearn.ensemble import VotingClassifier
#specify the estimators to use, specify anova first if it is used
estimators = [('ANOVA',anova),('XGB',best_xgb_model),('RF', best_rf_model)] 
voting_clf = VotingClassifier(estimators=estimators, voting='soft', weights=[1,1,1])
voting_clf.fit(X_train_scaled, y_train)

In [ ]:
def compute_feature_importance(voting_clf, weights):
    """ 
    Function to compute feature importance of ensemble feature selection 
    voting_clf = Voting Classifier object
    weights = weights used for each estimator 
    """
    
    feature_importance = dict()
    anova_featureimp = dict()

    try: # anova is not one of the estimators
        for est in voting_clf.estimators_:
            feature_importance[str(est)] = est.feature_importances_ #get the feature imp for each estimator
    except: #if anova is one of the estimators
        #get the feature imp for anova
        anova_featureimp[voting_clf.estimators_[0]] = voting_clf.estimators_[0].scores_/voting_clf.estimators_[0].scores_.sum()
        #get the feature imp for other estimators
        for est in voting_clf.estimators_[1:]: 
            feature_importance[str(est)] = est.feature_importances_ #get the feature imp for each estimator    
        #combine feature imp of anova and other estimators
        feature_importance.update(anova_featureimp)
    
    fe_scores = [0]*len(list(feature_importance.values())[0]) #initialize fe_scores as 0 
    for idx, imp_score in enumerate(feature_importance.values()): #idx:estimator index, imp_score: feature imp of est
        imp_score_with_weight = imp_score*weights[idx] #Multiply the weights of the base estimator to the importance score of each of the features.
        fe_scores = list(np.add(fe_scores, list(imp_score_with_weight)))
        
    return fe_scores

ensemble_fs = pd.DataFrame()
ensemble_fs['Feature'] = X_train_scaled.columns
ensemble_fs['Feature Importance'] = compute_feature_importance(voting_clf,weights = [1, 1, 1])
ensemble_fs = ensemble_fs.sort_values('Feature Importance', ascending=False)
ensemble_fs

In [ ]:
#take top n important features only in train and validation
n=200
features = ns_df.head(n)[["Feature"]] #choose the top n important features
X_train_scaled = X_train_scaled[features["Feature"]] #filter X_train_scaled with top n important features
X_val_scaled = X_val_scaled[features["Feature"]] #filter X_val_scaled with top n important features

In [ ]:
ensemble_fs.to_csv(featuresel_path+"Ensemble_Anova_XGB_RF.csv",index=False)

In [ ]:
# df2 = pd.read_csv(path+'DataPreparation/negative_parameters.csv')

In [ ]:
# df2.head()

In [ ]:
# df_IDV = df2.loc[:, df2.columns.str.startswith("IDV")]
# df_IDV.head()

In [ ]:
# df_IDV = df2.loc[:, df2.columns.str.startswith("IDV")]
# df_HVQK = df2.loc[:, df2.columns.str.startswith("HVQK")]
# df_IDV.min()
# df_HVQK.min()

In [ ]:
# df_HVQK = df2.loc[:, df2.columns.str.startswith("HVQK")]
# df_HVQK.head()

In [ ]:
# df_IDV.min()

In [ ]:
# df_HVQK.min()

In [ ]:
# df2.loc[:,((df2 == -5555).any()) & ((df2 == -9999).any())]

In [ ]:
# import pandas as pd
# path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/ML based speckle POC/"
# df2 = pd.read_csv(path+"SNR_R5_ww51.4.csv")
# df3 = df2[df2["VID"].isin(row100na_df["VID"])]
# df3 = df3.reset_index(drop=True)
# df3.shape
# df3.to_csv(path+"/DataPreparation/NA_rows_withsf.csv",index=False)

### Model Building

### a) SVM


In [ ]:
%%time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C':[0.1, 1, 10, 100, 1000], #Regularization parameter [0.1, 1, 10, 100, 1000]
    'kernel':['linear', 'poly', 'rbf', 'sigmoid'], #Specifies the kernel type to be used in the algorithm
    'gamma': ['scale', 'auto'],
    'class_weight': [None,'balanced'], #weights for unbalanced data     
    'random_state': [42]
}

# Grid search on SVM
classifier = SVC()
grid = GridSearchCV(estimator=classifier, param_grid=param_grid,scoring='balanced_accuracy',refit = True,n_jobs = -1,verbose=2,cv = 5)
grid_results = grid.fit(X_train_scaled,y_train) 

print("Best parameters:", grid_results.best_params_)

#model with best parameters
model = grid_results.best_estimator_ 
print(model)

# fs_importance= pd.DataFrame(list(zip(X_train_scaled.columns, model.feature_importances_)), columns=['Feature', 'Importance']).sort_values(by=['Importance'], ascending=False)  
# display(fs_importance)

In [ ]:
from joblib import load
grid_results= load(model_path+'SVMmodel.joblib') 

In [ ]:
#check grid search results
results_df = pd.DataFrame(grid_results.cv_results_)
results_df= results_df.drop(["mean_fit_time","std_fit_time","mean_score_time","std_score_time","split0_test_score","split1_test_score","split2_test_score","split3_test_score","split4_test_score"],axis=1)
results_df

In [ ]:
import joblib
from sklearn import metrics

#make folder for SVM if it doesn't exist
import os
model_path = path + 'ModelBuilding/SVM/' 
if not os.path.exists(model_path):
    os.makedirs(model_path)

#MODEL SAVING   
joblib.dump(model, model_path+"SVMmodel.joblib")

# Predicting the classes for validation set
y_pred = model.predict(X_val_scaled)

#overall accuracy
overall_acc = round(metrics.accuracy_score(y_val, y_pred)*100,2)
overall_acc = {'Overall Acc %':overall_acc}
overall_acc = pd.DataFrame([overall_acc])
overall_acc.to_csv(model_path+"Overall_Accuracy.csv")

#classification report
report = metrics.classification_report(y_val, y_pred,zero_division=0,output_dict=True)
report = pd.DataFrame(report).transpose()
report.to_csv(model_path+"Classification_Report.csv")

#confusion matrix with accuracies for each label
class_accuracies = []

for class_ in y_val.sort_values(ascending= True).unique():
    class_acc = round(np.mean(y_pred[y_val == class_] == class_)*100,2)
    class_accuracies.append(class_acc)
    
class_acc = pd.DataFrame(class_accuracies,index=['true:0', 'true:1'],columns= ["Accuracy %"])

cf_matrix = pd.DataFrame(
    metrics.confusion_matrix(y_val, y_pred, labels= [0, 1]), 
    index=['true:0', 'true:1'], 
    columns=['pred:0', 'pred:1']
)

ascend = None #input None/True/False to order the confusion matrix
if ascend == None:
    cf_matrix = pd.concat([cf_matrix,class_acc],axis=1)
else:
    cf_matrix = pd.concat([cf_matrix,class_acc],axis=1).sort_values(by=['Accuracy %'], ascending=ascend)

cf_matrix.to_csv(model_path+"Confusion_Matrix.csv",index=False)   

#validation results 
val_results = pd.concat([X_val_sf,X_val_scaled,pd.DataFrame(y_val),pd.DataFrame(y_pred,columns = ["PRED_SPECKLE"])],axis=1)
val_results.to_csv(model_path+"Val_results.csv",index=False) 


In [ ]:
y_val.value_counts()

In [ ]:
display(cf_matrix)
display(overall_acc)
display(report)

In [ ]:
# import numpy as np


# def plot_confusion_matrix(cm,
#                           target_names,
#                           title='Confusion matrix',
#                           cmap=None,
#                           normalize=True):
#     """
#     given a sklearn confusion matrix (cm), make a nice plot

#     Arguments
#     ---------
#     cm:           confusion matrix from sklearn.metrics.confusion_matrix

#     target_names: given classification classes such as [0, 1, 2]
#                   the class names, for example: ['high', 'medium', 'low']

#     title:        the text to display at the top of the matrix

#     cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
#                   see http://matplotlib.org/examples/color/colormaps_reference.html
#                   plt.get_cmap('jet') or plt.cm.Blues

#     normalize:    If False, plot the raw numbers
#                   If True, plot the proportions

#     Usage
#     -----
#     plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
#                                                               # sklearn.metrics.confusion_matrix
#                           normalize    = True,                # show proportions
#                           target_names = y_labels_vals,       # list of names of the classes
#                           title        = best_estimator_name) # title of graph

#     Citiation
#     ---------
#     http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

#     """
#     import matplotlib.pyplot as plt
#     import numpy as np
#     import itertools

#     accuracy = np.trace(cm) / np.sum(cm).astype('float')
#     misclass = 1 - accuracy

#     if cmap is None:
#         cmap = plt.get_cmap('Blues')

#     plt.figure(figsize=(8, 6))
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()

#     if target_names is not None:
#         tick_marks = np.arange(len(target_names))
#         plt.xticks(tick_marks, target_names, rotation=45)
#         plt.yticks(tick_marks, target_names)

#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


#     thresh = cm.max() / 1.5 if normalize else cm.max() / 2
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         if normalize:
#             plt.text(j, i, "{:0.4f}".format(cm[i, j]),
#                      horizontalalignment="center",
#                      color="white" if cm[i, j] > thresh else "black")
#         else:
#             plt.text(j, i, "{:,}".format(cm[i, j]),
#                      horizontalalignment="center",
#                      color="white" if cm[i, j] > thresh else "black")


#     plt.tight_layout()
#     plt.ylabel('Actual')
#     plt.xlabel('Predicted \naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
#     plt.show()

In [ ]:
# cm = metrics.confusion_matrix(y_val, y_pred)
# plot_confusion_matrix(cm=cm,target_names=[0,1],title='Confusion matrix', cmap=None,normalize=True)
                          